# Анализ таблиц JDWH - Прод vs Тест

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
pd.set_option('display.max_columns', 50)
pd.set_option('max_colwidth', 200)
#pd.set_option('display.max_rows', 500)

## Часть III. Индексы.

## 1. Загрузка и первичное преобразование данных

### Прод

In [3]:
prod = pd.read_csv('./data/indexes_prod_14122020.csv')

In [4]:
prod.head(3)

,schemaname,base_name,tablename,indexname,inds_count,ind,indexdef
0,dwh_bckp,amara_corr_overdues,amara_corr_overdues,pk_bckp_amara_corr_overdues,1,1,"CREATE UNIQUE INDEX pk_bckp_amara_corr_overdues ON dwh_bckp.amara_corr_overdues USING btree (id, insert_date)"
1,dwh_bckp,amara_rating_fitch,amara_rating_fitch,pk_bckp_amara_rating_fitch,1,1,"CREATE UNIQUE INDEX pk_bckp_amara_rating_fitch ON dwh_bckp.amara_rating_fitch USING btree (id, insert_date)"
2,dwh_bckp,amara_rating_fitch_forecast,amara_rating_fitch_forecast,pk_bckp_amara_rating_fitch_forecast,1,1,"CREATE UNIQUE INDEX pk_bckp_amara_rating_fitch_forecast ON dwh_bckp.amara_rating_fitch_forecast USING btree (id, insert_date)"


Добавим поле full_table_name

In [5]:
prod['full_table_name'] = prod['schemaname'] + '.' + prod['tablename']

Добавляем столбцы: признак уникальности (возможно это PK),тип индекска и его короткое определение

In [6]:
prod['uniq'] = prod.indexdef.apply(lambda x: 1 if 'CREATE UNIQUE INDEX' in x else 0)
prod['short_idx_def'] = prod.indexdef.apply(lambda x: x.split('USING')[1].strip())
prod['idx_type'] = prod.short_idx_def.apply(lambda x: x.split('(')[0].strip())
prod['ind_fields'] = prod.short_idx_def.apply(lambda x: x.split('(')[1].split(')')[0].strip())

In [7]:
prod.head(3)

,schemaname,base_name,tablename,indexname,inds_count,ind,indexdef,full_table_name,uniq,short_idx_def,idx_type,ind_fields
0,dwh_bckp,amara_corr_overdues,amara_corr_overdues,pk_bckp_amara_corr_overdues,1,1,"CREATE UNIQUE INDEX pk_bckp_amara_corr_overdues ON dwh_bckp.amara_corr_overdues USING btree (id, insert_date)",dwh_bckp.amara_corr_overdues,1,"btree (id, insert_date)",btree,"id, insert_date"
1,dwh_bckp,amara_rating_fitch,amara_rating_fitch,pk_bckp_amara_rating_fitch,1,1,"CREATE UNIQUE INDEX pk_bckp_amara_rating_fitch ON dwh_bckp.amara_rating_fitch USING btree (id, insert_date)",dwh_bckp.amara_rating_fitch,1,"btree (id, insert_date)",btree,"id, insert_date"
2,dwh_bckp,amara_rating_fitch_forecast,amara_rating_fitch_forecast,pk_bckp_amara_rating_fitch_forecast,1,1,"CREATE UNIQUE INDEX pk_bckp_amara_rating_fitch_forecast ON dwh_bckp.amara_rating_fitch_forecast USING btree (id, insert_date)",dwh_bckp.amara_rating_fitch_forecast,1,"btree (id, insert_date)",btree,"id, insert_date"


Убираем партиции

In [8]:
prod = prod[prod['schemaname'] + '.' + prod['tablename'] == prod['schemaname'] + '.' + prod['base_name']]
prod.sample(3)

,schemaname,base_name,tablename,indexname,inds_count,ind,indexdef,full_table_name,uniq,short_idx_def,idx_type,ind_fields
3838,dwh_dds,ref_deal_pledge,ref_deal_pledge,ref_deal_pledge_tmp_v2_dlpled_crncy_cd_idx,4,2,CREATE INDEX ref_deal_pledge_tmp_v2_dlpled_crncy_cd_idx ON dwh_dds.ref_deal_pledge USING btree (dlpled_prd_gid),dwh_dds.ref_deal_pledge,0,btree (dlpled_prd_gid),btree,dlpled_prd_gid
5355,dwh_stage,s01_m_ordpaydtl,s01_m_ordpaydtl,pk_s01_m_ordpaydtl,1,1,CREATE UNIQUE INDEX pk_s01_m_ordpaydtl ON dwh_stage.s01_m_ordpaydtl USING btree (nord),dwh_stage.s01_m_ordpaydtl,1,btree (nord),btree,nord
7940,dwh_stage,s02_cs_status_type,s02_cs_status_type,pk_s02_cs_status_type,1,1,CREATE UNIQUE INDEX pk_s02_cs_status_type ON dwh_stage.s02_cs_status_type USING btree (id),dwh_stage.s02_cs_status_type,1,btree (id),btree,id


==============================================================================================================

### Тест

In [9]:
test = pd.read_csv('./data/indexes_test_14122020.csv')

In [10]:
test.head(3)

,schemaname,base_name,tablename,indexname,inds_count,ind,indexdef
0,dwh_bckp,amara_rating_fitch,amara_rating_fitch,pk_bckp_amara_rating_fitch,1,1,"CREATE UNIQUE INDEX pk_bckp_amara_rating_fitch ON dwh_bckp.amara_rating_fitch USING btree (id, insert_date)"
1,dwh_bckp,amara_rating_fitch_forecast,amara_rating_fitch_forecast,pk_bckp_amara_rating_fitch_forecast,1,1,"CREATE UNIQUE INDEX pk_bckp_amara_rating_fitch_forecast ON dwh_bckp.amara_rating_fitch_forecast USING btree (id, insert_date)"
2,dwh_bckp,amara_rating_fitch_short,amara_rating_fitch_short,pk_bckp_amara_rating_fitch_short,1,1,"CREATE UNIQUE INDEX pk_bckp_amara_rating_fitch_short ON dwh_bckp.amara_rating_fitch_short USING btree (id, insert_date)"


Добавим поле full_table_name

In [11]:
test['full_table_name'] = test['schemaname'] + '.' + test['tablename']

Добавляем столбцы: признак уникальности (возможно это PK),тип индекска и его короткое определение

In [12]:
test['uniq'] = test.indexdef.apply(lambda x: 1 if 'CREATE UNIQUE INDEX' in x else 0)
test['short_idx_def'] = test.indexdef.apply(lambda x: x.split('USING')[1].strip())
test['idx_type'] = test.short_idx_def.apply(lambda x: x.split('(')[0].strip())
test['ind_fields'] = test.short_idx_def.apply(lambda x: x.split('(')[1].split(')')[0].strip())

In [13]:
test.sample(3)

,schemaname,base_name,tablename,indexname,inds_count,ind,indexdef,full_table_name,uniq,short_idx_def,idx_type,ind_fields
11360,dwh_stage,s01_t_trndtl_old,s01_t_trndtl_old_1_prt_70_2019_07_27,s01_t_trndtl_old_1_prt_70_2019_07_27_id_idx,4,4,CREATE INDEX s01_t_trndtl_old_1_prt_70_2019_07_27_id_idx ON dwh_stage.s01_t_trndtl_old_1_prt_70_2019_07_27 USING btree (id),dwh_stage.s01_t_trndtl_old_1_prt_70_2019_07_27,0,btree (id),btree,id
9918,dwh_stage,s01_t_trndtl,s01_t_trndtl_1_prt_78_2020_08_19,s01_t_trndtl_1_prt_78_2020_08_19_det_id_idx,4,2,CREATE INDEX s01_t_trndtl_1_prt_78_2020_08_19_det_id_idx ON dwh_stage.s01_t_trndtl_1_prt_78_2020_08_19 USING bitmap (det_id),dwh_stage.s01_t_trndtl_1_prt_78_2020_08_19,0,bitmap (det_id),bitmap,det_id
3052,dwh_dds,acc_transaction,acc_transaction_1_prt_210_2020_09_19,acc_transaction_1_prt_210_2020_09_19_trn_dt_acnt_gid_idx,4,2,CREATE INDEX acc_transaction_1_prt_210_2020_09_19_trn_dt_acnt_gid_idx ON dwh_dds.acc_transaction_1_prt_210_2020_09_19 USING btree (trn_dt_acnt_gid),dwh_dds.acc_transaction_1_prt_210_2020_09_19,0,btree (trn_dt_acnt_gid),btree,trn_dt_acnt_gid


Убираем партиции

In [14]:
test = test[test['schemaname'] + '.' + test['tablename'] == test['schemaname'] + '.' + test['base_name']]
test.sample(3)

,schemaname,base_name,tablename,indexname,inds_count,ind,indexdef,full_table_name,uniq,short_idx_def,idx_type,ind_fields
4083,dwh_dds,ref_anl_account,ref_anl_account,ref_anl_account_acnta_plancd_bmx,3,2,CREATE INDEX ref_anl_account_acnta_plancd_bmx ON dwh_dds.ref_anl_account USING bitmap (acnta_acntapln_cd),dwh_dds.ref_anl_account,0,bitmap (acnta_acntapln_cd),bitmap,acnta_acntapln_cd
11535,dwh_stage,s03_bsik,s03_bsik,s03_bsik_lifnr_idx,2,2,CREATE INDEX s03_bsik_lifnr_idx ON dwh_stage.s03_bsik USING btree (lifnr),dwh_stage.s03_bsik,0,btree (lifnr),btree,lifnr
6704,dwh_stage,s01_p_sys_std,s01_p_sys_std,pk_s01_p_sys_std,1,1,CREATE UNIQUE INDEX pk_s01_p_sys_std ON dwh_stage.s01_p_sys_std USING btree (id),dwh_stage.s01_p_sys_std,1,btree (id),btree,id


In [15]:
prod.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 581 entries, 0 to 7959
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   schemaname       581 non-null    object
 1   base_name        581 non-null    object
 2   tablename        581 non-null    object
 3   indexname        581 non-null    object
 4   inds_count       581 non-null    int64 
 5   ind              581 non-null    int64 
 6   indexdef         581 non-null    object
 7   full_table_name  581 non-null    object
 8   uniq             581 non-null    int64 
 9   short_idx_def    581 non-null    object
 10  idx_type         581 non-null    object
 11  ind_fields       581 non-null    object
dtypes: int64(3), object(9)
memory usage: 59.0+ KB


In [16]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 594 entries, 0 to 11540
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   schemaname       594 non-null    object
 1   base_name        594 non-null    object
 2   tablename        594 non-null    object
 3   indexname        594 non-null    object
 4   inds_count       594 non-null    int64 
 5   ind              594 non-null    int64 
 6   indexdef         594 non-null    object
 7   full_table_name  594 non-null    object
 8   uniq             594 non-null    int64 
 9   short_idx_def    594 non-null    object
 10  idx_type         594 non-null    object
 11  ind_fields       594 non-null    object
dtypes: int64(3), object(9)
memory usage: 60.3+ KB


### Сверяем только те таблицы, что есть и на Прод и на Тест

Прод

Удалим лишние столбцы

In [17]:
#Удаляем лишние столбцы
prod.drop(['base_name', 'short_idx_def'], inplace=True, axis = 1)#, 'indexdef'
prod.sample(3)

,schemaname,tablename,indexname,inds_count,ind,indexdef,full_table_name,uniq,idx_type,ind_fields
3923,dwh_dds,ref_sc_lot,ref_sc_lot_sqlt_sqtkt_gid_idx,10,9,CREATE INDEX ref_sc_lot_sqlt_sqtkt_gid_idx ON dwh_dds.ref_sc_lot USING btree (sqlt_sqtkt_gid),dwh_dds.ref_sc_lot,0,btree,sqlt_sqtkt_gid
5202,dwh_draft,amara_rating_hist,pk_amara_rating_hist,1,1,CREATE UNIQUE INDEX pk_amara_rating_hist ON dwh_draft.amara_rating_hist USING btree (id),dwh_draft.amara_rating_hist,1,btree,id
7939,dwh_stage,s02_cs_decision,pk_s02_cs_decision,1,1,CREATE UNIQUE INDEX pk_s02_cs_decision ON dwh_stage.s02_cs_decision USING btree (id),dwh_stage.s02_cs_decision,1,btree,id


Test

Удалим лишние столбцы

In [18]:
#Удаляем лишние столбцы
test.drop(['base_name', 'short_idx_def'], inplace=True, axis = 1)#, 'indexdef'
test.sample(3)

,schemaname,tablename,indexname,inds_count,ind,indexdef,full_table_name,uniq,idx_type,ind_fields
4052,dwh_dds,dict_orgform,dict_orgform_pk,1,1,CREATE UNIQUE INDEX dict_orgform_pk ON dwh_dds.dict_orgform USING btree (orgform_cd),dwh_dds.dict_orgform,1,btree,orgform_cd
6618,dwh_stage,s01_g_cli,s01_g_cli_ord_id_corp_idx,3,1,CREATE INDEX s01_g_cli_ord_id_corp_idx ON dwh_stage.s01_g_cli USING btree (ord_id) WHERE (typefl <> '1'::bpchar),dwh_stage.s01_g_cli,0,btree,ord_id
8330,dwh_stage,s01_t_pcn,s01_t_pcn_id_idx,2,1,"CREATE INDEX s01_t_pcn_id_idx ON dwh_stage.s01_t_pcn USING btree (id, specfl)",dwh_stage.s01_t_pcn,0,btree,"id, specfl"


### Выравниваем Прод и Тест

In [19]:
prod2 = prod[['full_table_name']]
prod2.head(2)

,full_table_name
0,dwh_bckp.amara_corr_overdues
1,dwh_bckp.amara_rating_fitch


In [20]:
test2 = test[['full_table_name']]
test2.head(2)

,full_table_name
0,dwh_bckp.amara_rating_fitch
1,dwh_bckp.amara_rating_fitch_forecast


Готовим список нужных имен таблиц (которые есть и на Прод и на Тест)

In [21]:
prod2_test2_merge = prod2.merge(test2, on='full_table_name', how='inner')
prod2_test2_merge.head(2)

,full_table_name
0,dwh_bckp.amara_rating_fitch
1,dwh_bckp.amara_rating_fitch_forecast


In [22]:
#Эти таблицы есть и на Прод и на Тест. Остальные будем удалять.
not_to_delete = list(prod2_test2_merge.full_table_name.value_counts().index)

In [23]:
not_to_delete[:10] #Первые 10 элементов

['dwh_dds.ref_sc_issue',
 'dwh_dds.ref_deal_forex',
 'dwh_dds.ref_sc_lot',
 'dwh_dds.ref_sc_ticket',
 'dwh_dds.ref_account',
 'dwh_dds.ref_deal_scissue',
 'dwh_dds.ref_gen_agreement',
 'dwh_dds.acc_sc_quote',
 'dwh_dds.ref_sc_rating',
 'dwh_dds.ref_pledge_object']

Теперь удаляем строки в Прод и Тест, не входящие в список по полю full_table_name

Прод

In [24]:
indices = prod[~(prod['full_table_name'].isin(not_to_delete))].index
prod.drop(indices, inplace=True)

Тест

In [25]:
indices = test[~(test['full_table_name'].isin(not_to_delete))].index
test.drop(indices, inplace=True)

### Теперь наборы данных Прод и Тест содержат одинаковые таблицы и готовы к сравнению инднксов

In [26]:
len(prod)

573

In [27]:
len(test)

566

Сравнение таблиц по количеству индексов:

In [28]:
tabs_different_number_of_indices = set()

In [29]:
def check_indexes_quantity(row):
    '''Сравнивает количество индексов для таблиц на Прод и Тест и
       если оно различно, добавляет таблицу во множество tabs_different_number_of_indices '''
    if row['inds_count'] != test[test['full_table_name'] == row['full_table_name']].inds_count.max():
        tabs_different_number_of_indices.add(row['full_table_name'])

In [30]:
prod.apply(lambda r: check_indexes_quantity(r), axis=1)

1       None
2       None
3       None
4       None
5       None
        ... 
7955    None
7956    None
7957    None
7958    None
7959    None
Length: 573, dtype: object

### Вот список таблиц, индексы которых будем сверять вручную:

In [31]:
list(tabs_different_number_of_indices)

['dwh_dds.ref_sc_rating',
 'dwh_dds.ref_sc_ticket',
 'dwh_stage.s01_ansignval',
 'dwh_dds.acc_doc_payment',
 'dwh_stage.s01_s_ordcash_add',
 'dwh_stage.s01_t_operjrn',
 'dwh_dds.ref_interbank_deals',
 'dwh_stage.s01_g_accblnhst',
 'dwh_dds.ref_anl_account',
 'dwh_dm.dm_transaction',
 'dwh_dds.ref_deal_scissue',
 'dwh_dds.ref_sc_issue',
 'dwh_stage.s01_t_deashdhst']

Теперь удалим строки с этими таблицами из Прод и Тест. Остальное можно попробовать сверить в автоматическом режиме

In [32]:
#Общий список таблиц для сверки вручную
tabs_to_manually_comparison = list(tabs_different_number_of_indices)

In [33]:
#Удаляем строки с этими таблицами:
indices = prod[prod['full_table_name'].isin(tabs_to_manually_comparison)].index
prod.drop(indices, inplace=True)

indices = test[test['full_table_name'].isin(tabs_to_manually_comparison)].index
test.drop(indices, inplace=True)

In [34]:
len(prod), len(test)

(502, 502)

**Видим, что на Прод и Тест осталось одинаковое количество строк!**

### Сверка всех индексов по определениям

In [35]:
tables_to_check = list(prod.full_table_name.value_counts().index)
tables_to_check

['dwh_dds.ref_deal_forex',
 'dwh_dds.ref_sc_lot',
 'dwh_dds.ref_account',
 'dwh_dds.acc_sc_quote',
 'dwh_dds.ref_gen_agreement',
 'dwh_dds.ref_pledge_object',
 'dwh_dds.ref_sc_identcode',
 'dwh_dds.ref_bank_corr_relation',
 'dwh_dds.ref_deal_current_account',
 'dwh_stage.s01_t_procmem',
 'dwh_stage.s01_ledacc_det',
 'dwh_dds.acc_transaction',
 'dwh_dds.ref_deal_pledge',
 'dwh_stage.s01_g_accbln',
 'dwh_stage.s01_t_trndtl',
 'dwh_stage.s01_s_ordcash',
 'dwh_dds.ref_deal_guarantees',
 'dwh_dds.ref_deal_letter_credit',
 'dwh_bckp.s01_led_acc_det_bkp',
 'dwh_stage.s01_g_cli',
 'dwh_dds.ref_pledge_link',
 'dwh_dds.ref_payment_card',
 'dwh_dds.acc_anl_account_balance',
 'dwh_dds.ref_client_corp',
 'dwh_stage.s01_s_monord',
 'dwh_dds.ref_pledge_obj_link',
 'dwh_stage.s01_l_mrtordobj',
 'dwh_stage.s01_l_mrtorddea',
 'dwh_stage.s01_t_procinh',
 'dwh_stage.s01_g_identdocdsc_std',
 'dwh_stage.s01_t_dea',
 'dwh_stage.s01_g_clirefval',
 'dwh_dds.acc_sc_deal_lot_link',
 'dwh_stage.s03_bsak',
 'dwh_s

In [36]:
print('Всего уникальных таблиц:', len(tables_to_check))

Всего уникальных таблиц: 348


In [37]:
tabs_diffrent_indexes_defs = set()

In [38]:
for tab in tables_to_check:
    sub_set_prod = prod[prod.full_table_name==tab]
    #display(sub_set_prod)
    
    sub_set_test = test[test.full_table_name==tab]
    #display(sub_set_test)
    
    if set(sub_set_prod.indexdef.value_counts().index) != set(sub_set_test.indexdef.value_counts().index):
        tabs_diffrent_indexes_defs.add(tab)
        
list(tabs_diffrent_indexes_defs)

['dwh_stage.s01_t_trndtl_tmp',
 'dwh_dm.dm_z10',
 'dwh_stage.s01_p_csrjrn',
 'dwh_stage.s01_p_ord',
 'dwh_dds.ref_account',
 'dwh_draft.amara_corr_overdues']

## Итого, нужно проверить следующие списки таблиц:

1. Таблицы с различным количеством индексов:

In [39]:
list(tabs_different_number_of_indices)

['dwh_dds.ref_sc_rating',
 'dwh_dds.ref_sc_ticket',
 'dwh_stage.s01_ansignval',
 'dwh_dds.acc_doc_payment',
 'dwh_stage.s01_s_ordcash_add',
 'dwh_stage.s01_t_operjrn',
 'dwh_dds.ref_interbank_deals',
 'dwh_stage.s01_g_accblnhst',
 'dwh_dds.ref_anl_account',
 'dwh_dm.dm_transaction',
 'dwh_dds.ref_deal_scissue',
 'dwh_dds.ref_sc_issue',
 'dwh_stage.s01_t_deashdhst']

2. Таблицы с различным определением индексов

In [40]:
list(tabs_diffrent_indexes_defs)

['dwh_stage.s01_t_trndtl_tmp',
 'dwh_dm.dm_z10',
 'dwh_stage.s01_p_csrjrn',
 'dwh_stage.s01_p_ord',
 'dwh_dds.ref_account',
 'dwh_draft.amara_corr_overdues']

**3. Итого, все аномальные таблицы**

In [41]:
tabs_to_manually_comparison += list(tabs_diffrent_indexes_defs)
tabs_to_manually_comparison

['dwh_dds.ref_sc_rating',
 'dwh_dds.ref_sc_ticket',
 'dwh_stage.s01_ansignval',
 'dwh_dds.acc_doc_payment',
 'dwh_stage.s01_s_ordcash_add',
 'dwh_stage.s01_t_operjrn',
 'dwh_dds.ref_interbank_deals',
 'dwh_stage.s01_g_accblnhst',
 'dwh_dds.ref_anl_account',
 'dwh_dm.dm_transaction',
 'dwh_dds.ref_deal_scissue',
 'dwh_dds.ref_sc_issue',
 'dwh_stage.s01_t_deashdhst',
 'dwh_stage.s01_t_trndtl_tmp',
 'dwh_dm.dm_z10',
 'dwh_stage.s01_p_csrjrn',
 'dwh_stage.s01_p_ord',
 'dwh_dds.ref_account',
 'dwh_draft.amara_corr_overdues']

In [42]:
len(tabs_to_manually_comparison)

19